In [1]:
import gc
import multiprocessing
from typing import Tuple, Any
!pip install pytorch-lightning

In [2]:
import numpy as np
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [3]:
from src.tools import *

In [4]:
torch.set_printoptions(precision=3, edgeitems=20, linewidth=250)
torch.set_float32_matmul_precision('high')
np.set_printoptions(precision=3, suppress=True, edgeitems=20, linewidth=250)
INT_BITS = 32
INT_MAX = (1 << (INT_BITS - 1)) - 1
INT_MIN = -(1 << (INT_BITS - 1))

In [5]:
class AND(nn.Module):
    def __init__(self):
        super().__init__()

        self.l1 = nn.Sequential(nn.Linear(2, 1), nn.ReLU())
        
        with torch.no_grad():
            self.l1[0].weight.fill_(1.0)
            self.l1[0].bias.fill_(-1.0)
        
        for layer in self.l1:
            if isinstance(layer, nn.Linear):
                layer.weight = nn.Parameter(layer.weight.double())
                layer.bias = nn.Parameter(layer.bias.double())

    def forward(self, x):
        return self.l1(x.double())

In [6]:
class OR(nn.Module):
    def __init__(self):
        super().__init__()

        self.l1 = nn.Sequential(nn.Linear(2, 1), nn.Hardtanh(max_val=1.0, min_val=-1.0))
        
        with torch.no_grad():
            self.l1[0].weight.fill_(1.0)
            self.l1[0].bias.fill_(0)
        
        for layer in self.l1:
            if isinstance(layer, nn.Linear):
                layer.weight = nn.Parameter(layer.weight.double())
                layer.bias = nn.Parameter(layer.bias.double())

    def forward(self, x):
        return self.l1(x.double())

In [56]:
class XOR(nn.Module):
    def __init__(self):
        super().__init__()

        self.vals = [AND(), OR()]
        
        for layer in self.vals:
            if isinstance(layer, nn.Linear):
                layer.weight = nn.Parameter(layer.weight.double())
                layer.bias = nn.Parameter(layer.bias.double())

    def forward(self, x):
        pred1 = self.vals[0](x)
        pred2 = self.vals[1](x)
        # print(torch.concatenate([torch.ones_like(pred1) - pred1, pred2], axis=-1))
        reshaped = torch.concatenate([torch.ones_like(pred1) - pred1, pred2], axis=-1)
        return self.vals[0](reshaped)

In [8]:
and1 = AND()
and1.forward(torch.tensor([[0, 0], [1, 0], [0, 1], [1, 1]], dtype=torch.float32))

tensor([[0.],
        [0.],
        [0.],
        [1.]], dtype=torch.float64, grad_fn=<ReluBackward0>)

In [9]:
xor = XOR()
list(xor.parameters())
xor.forward(torch.tensor([[0, 0], [1, 0], [0, 1], [1, 1]], dtype=torch.float32))

tensor([[0.],
        [1.],
        [1.],
        [0.]], dtype=torch.float64, grad_fn=<ReluBackward0>)

In [91]:
class SumBinModule(nn.Module):
    def __init__(self, n_bits = 32):
        super().__init__()
        self.XOR = XOR()
        self.AND = AND()
        self.OR = OR()
        
    def forward(self, x:  torch.Tensor):
        
        reshaped = x.reshape(x.shape[0], 2, x.shape[1] // 2)
        transposed = torch.transpose(reshaped, 1, 2)
        
        # after_and = reshaped.apply(self.AND)
        # after_xor = reshaped.apply(self.XOR)
        # print(after_and)
        # print(after_xor)
        after_and = self.AND(transposed)
        after_xor = self.XOR(transposed)
        print("After AND")
        print(after_and)
        after_roll = torch.roll(after_and, -1)
        print(after_roll)
        after_roll[:, -1, 0] = 0.0
        
        concat = torch.concatenate((after_roll, after_xor), axis=-1)
        print(concat)
        return torch.flatten(self.XOR(concat), start_dim=1)
        

In [92]:
summer = SumBinModule()
summer.forward(torch.tensor([[0, 0, 1,
                              1, 1, 1],
                             # [0, 1, 1, 0, 1, 1]
                             ], dtype=torch.float32))


After AND
tensor([[[0.],
         [0.],
         [1.]]], dtype=torch.float64, grad_fn=<ReluBackward0>)
tensor([[[0.],
         [1.],
         [0.]]], dtype=torch.float64, grad_fn=<RollBackward0>)
tensor([[[0., 1.],
         [1., 1.],
         [0., 0.]]], dtype=torch.float64, grad_fn=<CatBackward0>)


tensor([[1., 0., 0.]], dtype=torch.float64, grad_fn=<ReshapeAliasBackward0>)

In [12]:

class SumNaturalBinAutoEncoder(pl.LightningModule):
    def __init__(self, encoder: SumBinModule, loss_function=F.mse_loss, lr=3 * 1e-3):
        super().__init__()
        self.encoder = encoder
        self.loss_function = loss_function
        self.lr = lr

    def predict(self, x):
        return self.encoder(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.encoder(x)
        loss = self.loss_function(y_hat, y)
        return loss

    @staticmethod
    def _get_accuracy(y_hat, y) -> tuple[float, float]:
        eq = (y == torch.round(y_hat))
        return eq.all(axis=1).double().mean(), eq.double().mean()

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.encoder(x)
        loss = self.loss_function(y_hat, y)
        acc, bin_acc = self._get_accuracy(y_hat, y)
        metrics = {"test_acc": acc, "test_loss": loss, "test_bin_acc": bin_acc}
        self.log_dict(metrics)
        return metrics

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=2.5 * 1e-3)
        return optimizer

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.encoder(x)
        val_loss = self.loss_function(y_hat, y)
        acc, bin_acc = self._get_accuracy(y_hat, y)
        metrics = {"test_acc": acc, "test_loss": val_loss, "test_bin_acc": bin_acc}
        self.log_dict(metrics)
        return metrics

        # Return the validation loss and any other metrics you computed
        # return {'val_loss': val_loss, 'val_accuracy': accuracy}

In [13]:
from torch.utils.data import Dataset
import torch
import os
from tqdm import tqdm
import torch.random
import numpy as np
import pandas as pd

In [14]:
import warnings


class SimpleRandomNaturalBinSumDataset(Dataset):
    def __init__(self, size, transform=None, target_transform=None):
        self.size = size
        self.transform = transform
        self.target_transform = target_transform
        ints1 = np.random.randint(INT_MIN, INT_MAX, size=self.size, dtype=np.int32)
        ints2 = np.random.randint(INT_MIN, INT_MAX, size=self.size, dtype=np.int32)

        array1 = np.array([SimpleRandomNaturalBinSumDataset.decompose_to_bool_array(i) for i in tqdm(ints1)])
        array2 = np.array([SimpleRandomNaturalBinSumDataset.decompose_to_bool_array(i) for i in tqdm(ints2)])
        self.X = np.concatenate((array1.reshape((self.size, INT_BITS)),
                                 array2.reshape((self.size, INT_BITS))), axis=1)

        self.labels = np.array([SimpleRandomNaturalBinSumDataset.decompose_to_bool_array(i) for i in
                                tqdm((ints1 + ints2) % (1 << INT_BITS) + INT_MIN)])

    @staticmethod
    def compose_from_bool_array(bool_array):
        if not isinstance(bool_array, np.ndarray):
            raise ValueError("bool_array should be a numpy ndarray.")

        if bool_array.shape != (INT_BITS,):
            raise ValueError("bool_array should be a 1-dimensional array of size 32.")

        binary_str = "".join(str(int(bit)) for bit in bool_array)
        first_bit = binary_str[0]
        number_str = binary_str[1:]

        if first_bit == '0':
            return int(number_str, 2)
        else:
            return int(number_str, 2) + INT_MIN

    @staticmethod
    def decompose_to_bool_array(number):
        if not isinstance(number, np.int32) and not isinstance(number, np.int64) and not isinstance(number, int):
            raise ValueError(f"Number should be an integer. Received {type(number)}")

        if number < 0:
            return SimpleRandomNaturalBinSumDataset._decompose_positive_add_first_bit(number - INT_MIN, str(1))
        else:
            return SimpleRandomNaturalBinSumDataset._decompose_positive_add_first_bit(number, str(0))

    @staticmethod
    def _decompose_positive_add_first_bit(number, first_bit: str):
        binary_str = first_bit + bin(number)[2:].zfill(31)
        bool_array = np.array([int(bit) for bit in binary_str], dtype=np.float64)
        return bool_array

    def __len__(self):
        return self.size

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        values, labels = self.X[idx], self.labels[idx]

        if self.transform:
            values = self.transform(values)
        if self.target_transform:
            labels = self.target_transform(labels)
        return values, labels


In [15]:
class PyplineOfSunModule:
    def __init__(self, model: SumNaturalBinAutoEncoder):
        self.model = model

    def predict(self, x: int, y: int):
        expected = x + y
        x_arr = SimpleRandomNaturalBinSumDataset.decompose_to_bool_array(x)
        y_arr = SimpleRandomNaturalBinSumDataset.decompose_to_bool_array(y)
        model_input = torch.tensor(np.concatenate((x_arr, y_arr)))
        predictions = self.model.predict(model_input).detach().numpy().round()
        print(f"Prediction:\t{predictions}")
        print(f"Expected:\t{SimpleRandomNaturalBinSumDataset.decompose_to_bool_array(expected)}")
        return SimpleRandomNaturalBinSumDataset.compose_from_bool_array(predictions)


In [22]:
# pypline_of_sum = PyplineOfSunModule(torch.load(f"autoencoder-V03.pt"))
# pypline_of_sum.predict(np.int32(-3), np.int32(5))

FileNotFoundError: [Errno 2] No such file or directory: 'autoencoder-V03.pt'

In [17]:
# Parameters:
activation_fun = nn.Sigmoid()
# layer_sizes = [INT_BITS, 8, 8, 8, INT_BITS]
layer_sizes = [INT_BITS * 2, 512, 128, INT_BITS]
# layer_sizes = [INT_BITS * 2, 128, INT_BITS]
# layer_sizes = [1, 32, 32, 32, 1]

In [18]:
dataset = SimpleRandomNaturalBinSumDataset(1_000_000)
train_loader = DataLoader(dataset, batch_size=10_000,
                          num_workers=multiprocessing.cpu_count(),
                          )
valid_dataset = SimpleRandomNaturalBinSumDataset(100_000)
valid_loader = DataLoader(valid_dataset, batch_size=5_000,
                          num_workers=multiprocessing.cpu_count(),
                          )

100%|██████████| 100000/100000 [00:00<00:00, 173219.57it/s]


In [24]:
# model
autoencoder = SumNaturalBinAutoEncoder(SumBinModule(32))

In [20]:
test_dataset = SimpleRandomNaturalBinSumDataset(500_000)
test_loader = DataLoader(test_dataset, batch_size=1000, num_workers=multiprocessing.cpu_count())

100%|██████████| 500000/500000 [00:02<00:00, 181203.22it/s]


In [25]:
# train model
trainer = pl.Trainer(max_epochs=30)

# trainer.fit(autoencoder, train_dataloaders=train_loader)
trainer.test(autoencoder, test_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

torch.Size([1000, 64])


RuntimeError: shape '[500, 2]' is invalid for input of size 64000

In [ ]:
autoencoder.lr = 0.0025
# for i in range(5):
# print("#" * 40 + " " * 10 + f"Iteration № {i:5}" + " " * 10 + "#" * 40)
trainer = pl.Trainer(max_epochs=50)
trainer.fit(autoencoder, train_dataloaders=train_loader)
trainer.test(autoencoder, test_loader)

In [ ]:
torch.save(autoencoder, f"./models/autoencoder-[128-64]-V05.pt")

In [ ]:
# x_rand = torch.tensor(np.random.randint(low=0, high=2, size=(1, INT_BITS)).astype(np.float64))
# y_hat = autoencoder.predict(x_rand).round()
# print(x_rand == y_hat)

In [ ]:
# import plotly.express as px

In [ ]:
# print(x[0].astype(int))
# print(y[0])
# print(y[0].round().astype(int))

In [ ]:
# print(x_to_plot)
# print(y_to_plot)

In [ ]:
# fig = px.line(x=x_to_plot, y=y_to_plot)
# fig.show()
# 
# fig = px.line(x=x_to_plot, y=differ.sum(axis=1))
# fig.show()